# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [117]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [118]:

filepath= "output_data/cities.csv"
cities_df=pd.read_csv(filepath)
cities_df.dropna(inplace = True) 

cities_df

,City ID,City,Country,Lat,Long,Temperature,Percent Humidity,Clouds,Wind Speed,Max Temp
0,4032243,Vaini,TO,-21.2000,-175.2000,78.96,78,20,13.80,78.96
1,6170031,Tuktoyaktuk,CA,69.4541,-133.0374,26.01,92,99,4.29,26.01
2,3980614,Valle de Juárez,MX,19.9333,-102.9333,59.11,94,100,3.67,59.11
3,2162662,Inverell,AU,-29.7833,151.1167,70.16,28,99,4.00,70.16
4,2264557,Ponta do Sol,PT,32.6667,-17.1000,68.36,67,57,6.87,68.72
...,...,...,...,...,...,...,...,...,...,...
573,3893656,Copiapó,CL,-27.3667,-70.3333,53.29,56,0,2.13,53.29
574,1066831,Beroroha,MG,-21.6667,45.1667,71.06,40,0,2.51,71.06
575,2381334,Atar,MR,20.5169,-13.0499,83.80,21,59,5.97,83.80
576,1502073,Krasnosel'kup,RU,65.7000,82.4667,23.20,92,100,20.74,23.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [119]:
import gmaps
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [120]:
locations = cities_df[["Lat", "Long"]]
locations

humidity = cities_df["Percent Humidity"]

In [121]:

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1.0)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [122]:
#create new dataframe for narrowing down best weather conditions
best_cities_df = cities_df

#temp between 75 and 90 F
best_cities_df = best_cities_df.loc[(best_cities_df["Max Temp"] > 75) & (best_cities_df["Max Temp"] < 90)]

#windspeed under 5 mph
best_cities_df = best_cities_df.loc[best_cities_df["Wind Speed"] < 5]

#cloud cover less than 10%
best_cities_df = best_cities_df.loc[best_cities_df["Clouds"] < 10]

# Drop any rows with null values
best_cities_df = best_cities_df.dropna()


best_cities_df

,City ID,City,Country,Lat,Long,Temperature,Percent Humidity,Clouds,Wind Speed,Max Temp
14,2110227,Butaritari,KI,3.0707,172.7902,81.97,72,1,3.60,81.97
21,2446796,Bilma,NE,18.6853,12.9164,81.88,13,0,4.43,81.88
103,2141714,Bourail,NC,-21.5667,165.4833,79.83,53,0,4.70,79.83
211,1259395,Porbandar,IN,21.6422,69.6093,76.84,66,3,4.29,76.84
357,290030,Doha,QA,25.2867,51.5333,80.58,74,0,2.30,80.58
510,2447513,Arlit,NE,18.7369,7.3853,79.75,13,0,1.95,79.75
524,2155562,Nelson Bay,AU,-32.7167,152.1500,72.41,47,0,3.00,75.96
571,5477090,Lovington,US,32.9440,-103.3485,68.02,28,0,4.14,76.64


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [123]:
hotel_df=best_cities_df

In [124]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


#paramters to pass into request
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
   
    lat = row["Lat"]
    lng = row["Long"]

    params["location"] = f"{lat},{lng}"
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()

    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 14: Butaritari.
Closest hotel is Isles Sunset Lodge.
------------
Retrieving Results for Index 21: Bilma.
Closest hotel is Expeditions Ténére voyages.
------------
Retrieving Results for Index 103: Bourail.
Closest hotel is Motel Allamanda.
------------
Retrieving Results for Index 211: Porbandar.
Closest hotel is Lords Inn Porbandar.
------------
Retrieving Results for Index 357: Doha.
Closest hotel is Movenpick Hotel Doha.
------------
Retrieving Results for Index 510: Arlit.
Closest hotel is Case de Passage De la Prefecture.
------------
Retrieving Results for Index 524: Nelson Bay.
Closest hotel is Mantra Nelson Bay.
------------
Retrieving Results for Index 571: Lovington.
Closest hotel is Buena vida rv parks.
------------


In [125]:
hotel_df

,City ID,City,Country,Lat,Long,Temperature,Percent Humidity,Clouds,Wind Speed,Max Temp,Hotel Name
14,2110227,Butaritari,KI,3.0707,172.7902,81.97,72,1,3.60,81.97,Isles Sunset Lodge
21,2446796,Bilma,NE,18.6853,12.9164,81.88,13,0,4.43,81.88,Expeditions Ténére voyages
103,2141714,Bourail,NC,-21.5667,165.4833,79.83,53,0,4.70,79.83,Motel Allamanda
211,1259395,Porbandar,IN,21.6422,69.6093,76.84,66,3,4.29,76.84,Lords Inn Porbandar
357,290030,Doha,QA,25.2867,51.5333,80.58,74,0,2.30,80.58,Movenpick Hotel Doha
510,2447513,Arlit,NE,18.7369,7.3853,79.75,13,0,1.95,79.75,Case de Passage De la Prefecture
524,2155562,Nelson Bay,AU,-32.7167,152.1500,72.41,47,0,3.00,75.96,Mantra Nelson Bay
571,5477090,Lovington,US,32.9440,-103.3485,68.02,28,0,4.14,76.64,Buena vida rv parks


In [126]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]
locations

,Lat,Long
14,3.0707,172.7902
21,18.6853,12.9164
103,-21.5667,165.4833
211,21.6422,69.6093
357,25.2867,51.5333
510,18.7369,7.3853
524,-32.7167,152.1500
571,32.9440,-103.3485


In [127]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [128]:
#could not figure out how to remove old marker overlay from intial problem